*Odkomentuj i wykonaj ten kod, jeżeli masz problemy z zaimportowaniem bibliotek.*

In [2]:
# !pip install numpy==1.24.2
# !pip install seaborn==0.12.2
# !pip install pandas==1.5.3
# !pip install scikit-learn==1.2.1

In [3]:
import numpy as np
import seaborn as sns
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['figure.figsize'] = (20, 10)

In [4]:
df_web_local_paths = {
    'https://raw.githubusercontent.com/Kolo-Naukowe-Data-Science-PW/PoweR_2023/main/01_feature_engineering/Python/data/census_01.csv': 'census_data_01.csv',
    'https://raw.githubusercontent.com/Kolo-Naukowe-Data-Science-PW/PoweR_2023/main/01_feature_engineering/Python/data/census_02.csv': 'census_data_02.csv',
    'https://raw.githubusercontent.com/Kolo-Naukowe-Data-Science-PW/PoweR_2023/main/01_feature_engineering/Python/data/census_03.csv': 'census_data_03.csv',
    'https://raw.githubusercontent.com/Kolo-Naukowe-Data-Science-PW/PoweR_2023/main/01_feature_engineering/Python/data/census_04.csv': 'census_data_04.csv'
}

for web_path, local_path in df_web_local_paths.items():
    tmp = pd.read_csv(web_path)
    tmp.to_csv(local_path, index=False)

In [5]:
df = pd.read_csv('census_data_01.csv')

# PoweR

# Część I: Feature engineering
*Prowadzący: Dawid Płudowski*

### Podstawowe pojęcia 

* **ramka danych** - dane w postaci tabeli (u nas `df`)

* **obserwacja** - wiersz ramki danych; zawiera informacje o pojdeynczym wydarzeniu/fakcie (`np. df.iloc[1]`)

* **zmienna objaśniana** - kolumna ramki danych, którą chcemy przewidywać za pomocą modelu uczenia maszynowego (u nas `df.loc[:, 'income']`)

* **zmienne objaśniające** - kolumny tabeli, na podstawie których chcemy przewidywać zmienną objaśniającą (np. `df['education']`)

* **wartość brakująca** - komórka tabeli, która nie ma wartości; potocznie `null`, w Pythonie `None` (czasami kodowana za pomocą wartości specjalnych)




## Wartości brakujące

Szukamy w zbiorze danych wartości, które mogą kodować brak danych. Jak mogą wyglądać?

Zmienne kategoryczne:
* znaki zapytania
* puste napisy
* słowa-klucze, np. '*unknown*' , '*blank*'

Zmienne numeryczne:
* wartości spoza możliwego zakresu (np. 1000 w zmiennej `age`)
* wartość ujemna, jeżeli cała reszta jest dodatnia
* liczby, które często są stosowane do kodowania nulli, np. -999, 999

Po znalezieniu wartości, która reprezentuje wartość brakującą powinniśmy ją zakodować jako `None`, żeby później wygodnie z nią pracować.

**Uwaga**: biblioteka `pandas` oraz `numpy` posiadają własne oznaczenia na braki danych (`pd.NA`, `np.nan`, `np.NaN`), które nie są tym samym!

Artykuł o narzędziach do automatycznej analizy danych: [www.analyticsvidhya.com](https://www.analyticsvidhya.com/blog/2021/04/top-python-libraries-to-automate-exploratory-data-analysis-in-2021/)

In [6]:
%%capture

# przydatne funkcje

df['age'].hist() # wyświetla histogram wartości
df['occupation'].unique()
df[df['final_pop_weight'] < 0] # sposób na filtrowanie ramki danych
df['age'].mask(df['age'] > 80, None) # zamienia wartości, ktore spełniają warunek na None

### Zadanie 1. szukanie wartości brakujących w zbiorze `Census`

In [7]:
df.head()

,age,workclass,final_pop_weight,education,marital_status,occupation,race,sex,capital_gain,work_hours_per_week,native_country,income
0,90,?,77053,Highschool,Widowed,?,White,Female,0.000000,40,United-States,1.0
1,82,Private,132870,Highschool,Widowed,Exec-managerial,White,Female,42128.866677,18,United-States,1.0
2,66,?,186061,College,Widowed,?,Black,Female,0.000000,40,United-States,1.0
3,54,Private,140359,primary school,Divorced,Machine-op-inspct,White,Female,0.000000,40,United-States,1.0
4,41,Private,264663,College,Separated,Prof-specialty,White,Female,0.000000,40,United-States,1.0


In [8]:
df.describe()

,age,final_pop_weight,capital_gain,work_hours_per_week,income
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32538.000000
mean,110.297902,1.895912e+05,16011.616137,39.927121,0.759112
std,252.778636,1.056550e+05,36868.108854,14.344230,0.427629
min,17.000000,-1.230000e+02,0.000000,-1.000000,0.000000
25%,28.000000,1.177670e+05,0.000000,38.000000,1.000000
50%,39.000000,1.783120e+05,0.000000,40.000000,1.000000
75%,51.000000,2.369130e+05,11911.240526,45.000000,1.000000
max,999.000000,1.484705e+06,423844.271600,149.000000,1.000000


### Aktywność 1.
Czas: ok. 5min

Szukamy kolejnych braków danych w ramce i oznaczamy je jako `None`.

In [9]:
# miejsce na Twój kod :)

## Imputacja danych

Udało nam się zidentyfikować nulle w naszej ramce danych. Teraz musimy je uzupełnić, aby można na nich ćwiczyć model. Rodzaje braków danych możemy podzielić na następujące kategorie:
* Missing completely at random (**MCAR**)
* Missing at random (**MAR**)
* Missing not at random (nonignorable)

W przypadku naszych danych możemy założyć, że braki nie niosą ze sobą żadnej istotnej informacji (MCAR) i możemy je uzupełnić w konwencjonalny sposób. W przypadku danych numerycznych uzupełnimy je średnią, natomiast dane kategoryczne uzupełnimy najczęściej występującą wartością. Dlaczego taka strategia ma sens:
* jeżeli nic nie wiemy o naszym braku to, o ile pochodzi on z tego samego rozkładu co pozostałe dane, najmniej pomylimy się estymując go średnią/najczęściej występującą wartością

Artykuł o rodzajach braków danych: [displayr.com/different-types-of-missing-data/](https://www.displayr.com/different-types-of-missing-data/)

In [10]:
from sklearn.impute import SimpleImputer

In [11]:
%%capture

# przydatne funkcje
imputer_mean = SimpleImputer(missing_values=np.nan, strategy='mean') # zamienia NaN na średnią
imputer_mod = SimpleImputer(missing_values=np.nan, strategy='most_frequent') # zamienia NaN na najczęściej występującą wartość

imputer_mean.fit_transform(df[['capital_gain']]) 

### Zadanie 2.
Imputujemy braki danych

In [12]:
imputer_mean = SimpleImputer(strategy='mean')
new_age = imputer_mean.fit_transform(df[["age"]])
df['age'] = new_age

### Aktywność 2.
Czas: ok. 3 min

Kontynuujemy uzupełniane nulli.

In [13]:
# miejsce na twój kod :)

## Szukanie outlierów

Nasze dane są już kompletne, jednak niektóre z nich, nawet jeżeli prawdziwe, mogą utrudnić modelowi proces uczenia. Dlaczego tak się dzieje?

### Wartości odstające 
Nasz zbiór danych zawiera pojedyncze obseracje, które zaburzają przewidywanie.

**Przykład**:

Dysponujemy informacją o zawodzie (lub jego braku) grupy osób i chcemy przewidywać na podstawie tego wielkość ich oszczędności. Logiczne jest, że osoba bezrobotna powinna mieć ich raczej mniej niż np. data scientist. Jeżeli jednak w naszej grupie osób na 1.000 bezrobotnych jeden jest miliarderem (np. odziedziczył te pieniądze) to czy taka informacja nie jest myląca? Niektóre modele będą próbować uogólnić ten fakt i nauczą się, że skoro 999 osób z danej grupy ma 0zł oszczędności, a jedna 1.000.000.000zł to średnio osoba bezrobotna ma 1.000.000zł oszczędności.

Z drugiej strony, usunięcie podpopulacji miliarderów z danych sprawi, że nasz model nigdy nie nauczy się jak ich przewidywać.

### Generalizacja

Nigdy nie dysponujemy kompletnymi danymi o danym zjawisku, dlatego też chcielibyśmy, aby nasz model generalizował fakty. Tak jak w powyższym przykładzie, powiedzielibyśmy: "oszczędności ludzi bezrobotnych różnią się w zależności od innych czynników, ale generalnie są niskie".

### Małe grupy

Czasami dana cecha (np. bycie noblistą z fizyki) jest tak rzadka, że nie mamy wystarczająco dużo obserwacji, które by ją posiadały, aby móc model mógł na jej podstawie wyciągnąc wniosków. Czasami decydujemy się, aby takie małe podpopulacje łączyć ze sobą, w szczególności jeżeli podejrzewamy, że nie wykazują znaczących róznic (np. nobliści z fizyki i ekonomii). 

### Zadanie 3. 

Szukamy i usuwamy outliery ze zbioru danych


In [14]:
#

### Aktywność 3.
Czas: ok. 10min

Kontynuujemy poszukiwania outlierów. Istnieją różne strategie radzenia sobie z nimi, my wykorzystamy najprostszą, to znaczy "na oko" ;) Użyjemy w tym celu wykresów, aby zdecydować, gdzie "obciąć" wartości numeryczne oraz jak zgrupować wartości kategoryczne. 

In [15]:
df = pd.read_csv('./census_data_02.csv')

In [16]:
%%capture

# przydatne funkcje
sns.countplot(data=df, x='education') # wykres dla zmiennych kategorycznych
sns.histplot(data=df, x='age', bins=100) # wykres dla zmiennych numerycznych

In [17]:
# miejsce na Twój kod :)

## Kodowanie zmiennych kategorycznych

Kolejnym krokiem w przygotowywaniu naszych danych jest zamiana zmiennych kategorycznych na liczbowe. Dlaczego taka zmiana jest potrzebna?
* modele uczenia maszynowego to skomplikowane funkcje matematyczne - potrzebują liczb jako argumentów
* czasami zmienne kateogryczne mają swój naturalny porządek (np. mały - średni - duży); odpowiednie kodowanie dostarczy modelowi informacji o tym porządku
* popularne biblioteki do uczenia maszynowego w **Pythonie** (np. używany na tych warsztatach `sklearn`) zwykle nie wspierają wartości kategorycznych


In [18]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [19]:
%%capture

# Przydatne funkcje:

df["education"].unique()  # tabela["zmienna"].unique() ~ zwraca unikalne wartości w danej kolumnie

onehot_encoder = OneHotEncoder(sparse_output=False) # 
new_columns = onehot_encoder.fit_transform(df[["education"]]) # zwraca zakodowaną kolumnę
new_columns_names = onehot_encoder.get_feature_names_out() # zwraca nazwy 

ordinal_encoder = OrdinalEncoder(
    # categories=[['cat_01', 'cat_02', 'cat_03', '...']]
    ) # param categories ustala porządek kodowania
ordinal_encoder.fit_transform(df[["education"]])
ordinal_encoder.get_feature_names_out()

df_temp = pd.DataFrame(data=new_columns, columns=new_columns_names) # tworzy nową tabelę

df.drop(columns=["education"]) # usuwa z tabeli kolumnę
df.join(df_temp) # łączy dwie tabele 

### Zadanie 4. kodowanie zbioru `Census`

In [20]:
df.head()

,age,workclass,final_pop_weight,education,marital_status,occupation,race,sex,capital_gain,work_hours_per_week,native_country,income
0,90.0,NaN,77053,Highschool,Widowed,?,White,Female,0.000000,40,United-States,1.0
1,82.0,Private,132870,Highschool,Widowed,Exec-managerial,White,Female,628.643586,18,United-States,1.0
2,66.0,NaN,186061,College,Widowed,?,Black,Female,0.000000,40,United-States,1.0
3,54.0,Private,140359,primary school,Divorced,Machine-op-inspct,White,Female,0.000000,40,United-States,1.0
4,41.0,Private,264663,College,Separated,Prof-specialty,White,Female,0.000000,40,United-States,1.0


### Aktywność 4.
Czas: ok. 7min

Kodujemy pozostałe zmienne kategoryczne. Metodę kodowania (`OrdinalEncoder`, `OneHotEncoder`) wybieramy zgodnie z poznanymi wcześniej praktykami. 

In [21]:
df = pd.read_csv('./census_data_03.csv')

In [22]:
# miejsce na twój kod :)

## Transformacje zmiennych

Mamy już ramkę danych posiadającą tylko zmienne liczbowe, bez nulli i z obciętymi outlierami. Ramka w takiej postaci jest już gotowa do użycia jej w celu uczenia modelu. Mimo to, nadal może nasze dane ulepszyć.

#### Po co transformujemy zmienne?

* niektóre modele uczenia maszynowego powstały w oparciu o matematyczne założenia jakie powinny spełniać dane. W szczególności, częstym założeniem jest to, że pojdencze zmienne pochodzą z rozkładu normalnego
* zrzutowanie zmiennych do tego samego przedziału lub kształu pozwala nam później łatwo porównywać ich znaczenie w procesie *XAI* ([artykuł o metodach XAI](https://ema.drwhy.ai/))

#### Których zmiennych nie transformujemy?

* zmienne, które są zakodowanymi wartościami kategorycznymi nie potrzebują kolejnych transformacji.

![rozkład normalny](https://cdn.corporatefinanceinstitute.com/assets/normal-distribution.png)

In [23]:
from sklearn.preprocessing import QuantileTransformer

In [ ]:
df.hist(figsize=(10, 10))

In [25]:
normal_transformer = QuantileTransformer(output_distribution='normal')
df['age'] = normal_transformer.fit_transform(df[['age']])

### Aktywność 5.
Czas: ok. 5min

Przeprowadzamy transformacje zmiennych do rozkładu normalnego.

In [26]:
df = pd.read_csv('./census_data_04.csv')

In [27]:
# miejsce na Twój kod :) 

### Aktywność dodatkowa - zaawansowane kodowanie zmiennych kategorycznych

Przechodzimy do zbioru **taxis**. Poniżej znajduje się kod, który mapuje dni tygodnia na liczby od 0 do 6. Chcemy je zakodować jako zmienną cykliczną; aby to zrobić wykonamy następujące kroki:
* przeskalujmy zmienne do zakresu $[0, 2\pi]$, np. mnożąc liczby przez $\frac{2\pi}{6}$ (dlaczego przez 6?)
* obliczymy $\sin$ i $\cos$ dla tak przeskalowanych zmiennych
* na koniec wystarczy dodać kolumny z $\sin$ i $\cos$ do tabeli i usunąć starą zmienną reprezentującą dni tygodnia :)

Poniżej znajduje się kod, który zwizualizuje twoje wyniki, sprawdź, czy wygenerowane punkty znajdują się na okręgu!

In [28]:
df_taxis = sns.load_dataset("taxis")
df_taxis['pickup'] = pd.to_datetime(df_taxis['pickup']).dt.weekday
df_taxis.head()

,pickup,dropoff,passengers,distance,fare,tip,tolls,total,color,payment,pickup_zone,dropoff_zone,pickup_borough,dropoff_borough
0,5,2019-03-23 20:27:24,1,1.60,7.0,2.15,0.0,12.95,yellow,credit card,Lenox Hill West,UN/Turtle Bay South,Manhattan,Manhattan
1,0,2019-03-04 16:19:00,1,0.79,5.0,0.00,0.0,9.30,yellow,cash,Upper West Side South,Upper West Side South,Manhattan,Manhattan
2,2,2019-03-27 18:00:25,1,1.37,7.5,2.36,0.0,14.16,yellow,credit card,Alphabet City,West Village,Manhattan,Manhattan
3,6,2019-03-10 01:49:51,1,7.70,27.0,6.15,0.0,36.95,yellow,credit card,Hudson Sq,Yorkville West,Manhattan,Manhattan
4,5,2019-03-30 13:37:14,3,2.16,9.0,1.10,0.0,13.40,yellow,credit card,Midtown East,Yorkville West,Manhattan,Manhattan


In [29]:
%%capture

# przydatne funkcje i atrubuty

np.pi # wartość pi
np.cos(2) 
np.sin(2 * np.pi)

df_taxis['pickup'] * 2 * np.pi - 5 # możemy mnożyć całą kolumnę przez stałą

In [30]:
# miejsce na Twoje rozwiązanie :)

In [ ]:
# kod do wizualizacji

sns.scatterplot(x=cos_column, y=sin_column)
plt.ylim(-2, 2)
plt.xlim(-2, 2)

## Aktywność dodatkowa - budowa pipelinów

Zrobiliśmy w trakcie tych warsztatów wiele modyfikacji naszych danych. Chcielibyśmy posiadać narzędzie, które te wszystkie zmiany wykona w ramach jednej funkcji, np. gdybyśmy chcieli analogiczne zmiany zaaplikować do zbioru testowego albo nowo dostarczonych danych. Funkcje, które realizuje ten cel to tzw. `pipeline`'y. W ramach ostatniej sekcji naszych warsztatów spróbujemy zaimplementować klasę, która wykona wszystkie wcześniej wspomniane transformacje za pomocą jednej metody `.transform(dataset)`.

In [ ]:
# 

#### Przydatne linki
* [`miceforest`](https://pypi.org/project/miceforest/)
* [`pycircural`](https://towardsdatascience.com/introducing-pycircular-a-python-library-for-circular-data-analysis-bfd696a6a42b)
* [`feature-engine`](https://feature-engine.trainindata.com/en/latest/)